# Marine eutrophication impact factors 

Exiobase categories:
- N - agriculture - water

LC-Impact stressors
- Marine eutrophication

In [1]:
# import required libraries
import pymrio
import numpy as np
import pandas as pd
import pycountry as pyc

In [2]:
# exiobase 2011 is used for calculating share of stressor for each region-product pair
exio3_11 = pymrio.parse_exiobase3(path="/Users/tterimaa/code/projects/impact-factors/IOT_2011_pxp.zip")
# exiobase 2019 is used for impact factors
exio3_19 = pymrio.parse_exiobase3(path="/Users/tterimaa/code/projects/impact-factors/IOT_2019_pxp")

## Calculate DRorigin
DRorigin is a matrix which describes the amount of the driver of biodiversity loss (DR) that occurs in impact region i sector k and is driven by consumption in region j sector k.

To calculate the matrix we need to
1. Aggregate relevant drivers from exiobase together
2. Diagonalize the aggregated driver and re-calculate the IO system

In [ ]:
# no need for aggregation

In [3]:
# diagonalize stressor
# optimized for speed
diag_water = exio3_11.satellite.diag_stressor(("N - agriculture - water"))

# First, make sure that L matrix is calculated 
if exio3_11.L is None:
    # try loading the L matrix from pickles/exio3_11_L.pickle
    try:
        exio3_11.L = pd.read_pickle("pickles/exio3_11_L.pickle")
        print("L matrix loaded from pickle")
    except FileNotFoundError:
        print("L matrix not found, calculating it from scratch.")
        print("Calculating A")
        exio3_11.A = pymrio.calc_A(exio3_11.Z, exio3_11.x)
        print("Calculating L")
        exio3_11.L = pymrio.calc_L(exio3_11.A)
        # save the L matrix to a pickle file
        pd.to_pickle(exio3_11.L, "pickles/exio3_11_L.pickle")
else:
    print("L already loaded")

# calculate S (direct emission multipliers)
if diag_water.S is None:
    print("Calculating S")
    diag_water.S = pymrio.calc_S(diag_water.F, exio3_11.x)

Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()

diag_water.D_cba, _, _, _ = pymrio.calc_accounts(diag_water.S, exio3_11.L, Y_agg)
diag_water.D_cba

L matrix loaded from pickle
Calculating S


/private/tmp/nix-shell-97002-0/ipykernel_58859/39903974.py:27: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = exio3_11.Y.groupby(level="region", axis=1, sort=False).sum()


region                                                              AT  \
sector                                                      Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              6.402791e-03   
       Cereal grains nec                                  2.142313e-02   
       Vegetables, fruit, nuts                            2.720965e-04   
       Oil seeds                                          2.384799e-03   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       6.451847e-05   
       Recreational, cultural and sporting services (92)  1.882675e-03   
       Other services (93)                                1.580414e-04   
       Private households with employed persons (95)      5.587061e-07   
       Extra-territorial organizations and bodies         0.000000e+00   

region                                                                   \
sector                                                            Wheat   
region sector                                                             
AT     Paddy rice                                              0.000000   
       Wheat                                              608974.564598   
       Cereal grains nec                                      23.602639   
       Vegetables, fruit, nuts                                 1.168402   
       Oil seeds                                               5.970210   
...                                                                 ...   
WM     Membership organisation services n.e.c. (91)            0.008239   
       Recreational, cultural and sporting services (92)       0.286583   
       Other services (93)                                     0.016363   
       Private households with employed persons (95)           0.000088   
       Extra-territorial organizations and bodies              0.000000   

region                                                                      \
sector                                                   Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  8.360758e+01   
       Cereal grains nec                                      4.098168e+06   
       Vegetables, fruit, nuts                                6.849809e+00   
       Oil seeds                                              2.958343e+01   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           2.722233e-02   
       Recreational, cultural and sporting services (92)      1.078214e+00   
       Other services (93)                                    5.557015e-02   
       Private households with employed persons (95)          3.109801e-04   
       Extra-territorial organizations and bodies             0.000000e+00   

region                                                                            \
sector                                                   Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        7.244308e+02   
       Cereal grains nec                                            1.356209e+03   
       Vegetables, fruit, nuts                                      3.079774e+06   
       Oil seeds                                                    2.068045e+02   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)            

## Calculate DR share
DR share is a new matrix that represents the share of the driver in the impact region i from the total amount of driver that is driven by consumption in region j sector k.

To calculate the matrix each column of DR origin is shared by the sum of that column.

In [4]:
# calculate dr_s - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
columns = {}
for series_name, series in diag_water.D_cba.items():
    series_sum = series.sum()
    columns[series_name] = series / series_sum

dr_s_w = pd.DataFrame(columns)
dr_s_w

AT  \
                                                            Paddy rice   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              3.278069e-07   
       Cereal grains nec                                  1.096811e-06   
       Vegetables, fruit, nuts                            1.393066e-08   
       Oil seeds                                          1.220957e-07   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       3.303184e-09   
       Recreational, cultural and sporting services (92)  9.638824e-08   
       Other services (93)                                8.091322e-09   
       Private households with employed persons (95)      2.860435e-11   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                        \
                                                                 Wheat   
region sector                                                            
AT     Paddy rice                                         0.000000e+00   
       Wheat                                              2.588195e-01   
       Cereal grains nec                                  1.003133e-05   
       Vegetables, fruit, nuts                            4.965812e-07   
       Oil seeds                                          2.537391e-06   
...                                                                ...   
WM     Membership organisation services n.e.c. (91)       3.501850e-09   
       Recreational, cultural and sporting services (92)  1.218002e-07   
       Other services (93)                                6.954595e-09   
       Private households with employed persons (95)      3.727299e-11   
       Extra-territorial organizations and bodies         0.000000e+00   

                                                                            \
                                                         Cereal grains nec   
region sector                                                                
AT     Paddy rice                                             0.000000e+00   
       Wheat                                                  1.010947e-05   
       Cereal grains nec                                      4.955330e-01   
       Vegetables, fruit, nuts                                8.282498e-07   
       Oil seeds                                              3.577103e-06   
...                                                                    ...   
WM     Membership organisation services n.e.c. (91)           3.291608e-09   
       Recreational, cultural and sporting services (92)      1.303731e-07   
       Other services (93)                                    6.719306e-09   
       Private households with employed persons (95)          3.760240e-11   
       Extra-territorial organizations and bodies             0.000000e+00   

                                                                                  \
                                                         Vegetables, fruit, nuts   
region sector                                                                      
AT     Paddy rice                                                   0.000000e+00   
       Wheat                                                        7.846721e-05   
       Cereal grains nec                                            1.468987e-04   
       Vegetables, fruit, nuts                                      3.335877e-01   
       Oil seeds                                                    2.240017e-05   
...                                                                          ...   
WM     Membership organisation services n.e.c. (91)                 1.327250e-07   
       Recreational, cultural and sporting services (92)     

## Calculate DR unit
DR unit is a region harmonized version of DR share.

To calculate DR unit we need to:
1. Identify regions that are missing from exiobase (rest of the world regions) but are present in lc-impact.
2. Assume that the impacts are divided evenly across the rest of the world category
3. By that assumption we can add the missing countries to DR share as the impact of the row region where country belongs to divided by the number of countries in that region

In [76]:
# harmonize regions 

# for this harmonization we need to know which countries go into:
# 1. rest of the world europe
# 2. rest of the world asia pacific
# 3. rest of the world middle east
# 4. rest of the world africa
# 5. rest of the world americas

row_eu_countries = {
    'AL': 'Albania',
    'AD': 'Andorra',
    'AZ': 'Azerbaijan',
    'BA': 'Bosnia and Herzegovina',
    'BY': 'Belarus',
    'FO': 'Faroe Islands',
    'GI': 'Gibraltar',
    'GG': 'Guernsey',
    'IS': 'Iceland',
    'JE': 'Jersey',
    'GL': 'Greenland',
    'LI': 'Liechtenstein',
    'MC': 'Monaco',
    'GE': 'Georgia',
    'MK': 'Macedonia',
    'MD': 'Moldova',
    'ME': 'Montenegro',
    'SJ': 'Svalbard and Jan Mayen',
    'UA': 'Ukraine',
}

row_asia_pacific_countries = {
    'AS': 'American Samoa', 'BD': 'Bangladesh', 'BN': 'Brunei', 'BT': 'Bhutan', 'CC': 'Cocos (Keeling) Islands', 'CK': 'Cook Islands', 'CX': 'Christmas Island',
    'KH': 'Cambodia', 'FJ': 'Fiji', 'FM': 'Micronesia', 'GU': 'Guam', 'IO': 'British Indian Ocean Territory (the)',
    'KZ': 'Kazakhstan', 'KI': 'Kiribati', 'KG': 'Kyrgyzstan', 'LA': 'Laos', 'MY': 'Malaysia', 'MH': 'Marshall Islands', 'MP': 'Northern Mariana Islands', 'MV': 'Maldives',
    'MN': 'Mongolia', 'MM': 'Myanmar (Burma)', 'MO': 'Macao', 'NR': 'Nauru', 'NU': 'Niue', 'NF': 'Norfolk Island', 'NC': 'New Caledonia', 'NZ': 'New Zealand',
    'KP': 'North Korea', 'PW': 'Palau', 'PK': 'Pakistan', 'PG': 'Papua New Guinea', 'PH': 'Philippines', 'PN': 'Pitcairn', 'PF': 'French Polynesia',
    'WS': 'Samoa', 'SB': 'Solomon Islands', 'LK': 'Sri Lanka', 'SG': 'Singapore', 'TK': 'Tokelau',
    'TJ': 'Tajikistan', 'TH': 'Thailand', 'TO': 'Tonga', 'TM': 'Turkmenistan', 'TV': 'Tuvalu', 'WF': 'Wallis and Futuna',
    'VU': 'Vanuatu', 'VN': 'Vietnam'
}

row_african_countries = {
    'DZ': 'Algeria', 'AO': 'Angola', 'BJ': 'Benin', 'BW': 'Botswana', 'BF': 'Burkina Faso', 'BI': 'Burundi',
    'CM': 'Cameroon', 'CG': 'Congo', 'CV': 'Cape Verde', 'CF': 'Central African Republic', 'GF': 'French Guiana', 'TD': 'Chad', 'KM': 'Comoros',
    'DJ': 'Djibouti', 'EG': 'Egypt', 'GQ': 'Equatorial Guinea', 'ER': 'Eritrea', 'EH': 'Western Sahara',
    'ET': 'Ethiopia', 'GA': 'Gabon', 'GM': 'Gambia, The', 'GH': 'Ghana', 'GN': 'Guinea', 'GW': 'Guinea-Bissau',
    'CI': 'Ivory Coast', 'KE': 'Kenya', 'KY': 'Cayman Islands', 'LS': 'Lesotho', 'LR': 'Liberia', 'LY': 'Libya', 'MG': 'Madagascar',
    'MW': 'Malawi', 'ML': 'Mali', 'MR': 'Mauritania', 'MU': 'Mauritius', 'MA': 'Morocco', 'MZ': 'Mozambique', 'MQ': 'Martinique',
    'NA': 'Namibia', 'NE': 'Niger', 'NG': 'Nigeria', 'RE': 'Reunion',  'RW': 'Rwanda', 'ST': 'São Tomé and Príncipe', 'SN': 'Senegal', 'SH': 'Saint Helena',
    'SL': 'Sierra Leone', 'SO': 'Somalia', 'SC': 'Seychelles',
    'SD': 'Sudan', 'TZ': 'Tanzania', 'TG': 'Togo', 'TN': 'Tunisia', 'UG': 'Uganda', 'YT': 'Mayotte', 'ZM': 'Zambia'
}

row_american_countries = {
    'AQ': 'Antarctica', 'AI': 'Anguilla', 'AG': 'Antigua and Barbuda', 'AW': 'Aruba', 'AR': 'Argentina', 'BS': 'Bahamas, The', 'BZ': 'Belize', 'BO': 'Bolivia',
    'BB': 'Barbados', 'BM': 'Bermuda', 'BV': 'Bouvet Island', 'CL': 'Chile', 'CO': 'Colombia', 'CR': 'Costa Rica', 'CU': 'Cuba', 'DO': 'Dominican Republic', 'DM': 'Dominica',
    'EC': 'Ecuador', 'FK': 'Falkland Islands (Islas Malvinas)', 'SV': 'El Salvador', 'GD': 'Grenada', 'GT': 'Guatemala', 'GY': 'Guyana', 'GP': 'Guadeloupe',
    'HT': 'Haiti', 'HN': 'Honduras', 'JM': 'Jamaica',
    'MS': 'Montserrat', 'NI': 'Nicaragua', 'PA': 'Panama', 'PE': 'Peru', 'PR': 'Puerto Rico', 'PM': 'Saint Pierre and Miquelon', 'KN': 'Saint Kitts and Nevis', 'LC': 'Saint Lucia',
    'SR': 'Suriname', 'TC': 'Turks and Caicos Islands', 'TT': 'Trinidad and Tobago', 'UY': 'Uruguay', 'VE': 'Venezuela', 
    'VC': 'Saint Vincent and the Grenadines', 'VG': 'Virgin Islands (British)', 'VI': 'Virgin Islands (U.S.)'
}

row_middle_eastern_countries = {
    'AF': 'Afghanistan', 'AM': 'Armenia', 'BH': 'Bahrain',
    'IR': 'Iran', 'IQ': 'Iraq', 'IL': 'Israel', 'JO': 'Jordan', 'KW': 'Kuwait', 'LB': 'Lebanon', 'OM': 'Oman',
    'QA': 'Qatar', 'SA': 'Saudi Arabia', 'SY': 'Syria', 'AE': 'United Arab Emirates',
    'YE': 'Yemen'
}

exio_regions = exio3_11.get_regions()
row_regions = {
    "WA": "Asia and pacific",
    "WE": "Europe",
    "WF": "Africa",
    "WM": "Middle east",
    "WL": "America"
}

# harmonize regions between lc-impact and exiobase
all_row_region_keys = list(row_eu_countries.keys()) + list(row_asia_pacific_countries.keys()) + list(row_african_countries.keys()) + list(row_american_countries.keys()) + list(row_middle_eastern_countries.keys())
exio_regions_without_row = [region for region in exio_regions if region not in row_regions.keys()]
print(len(exio_regions_without_row)) # exiobase contains 44 exact regions + 5 row regions

# combine exact regions from exiobase with exact row regions
all_regions = list(all_row_region_keys) + list(exio_regions_without_row)
print("Regions: ", len(all_regions))
print("Unique regions: ", len(set(all_regions)))
assert len(all_regions) == len(set(all_regions)), "There are duplicate regions in the all_regions list"

44
Regions:  225
Unique regions:  225


In [ ]:
# load and prepare lc-impact data
lci = pd.read_excel("/Users/tterimaa/code/projects/impact-factors/lc-impact/LC-Impact/9-marine eutrophication/CFs_marine_eutrophication.xlsx", sheet_name="country CFs")
lci = lci[["Country.1", "CF for direct N emission to marine system [PDF*yr/kg]"]]
lci.rename(columns={lci.columns[0]: "Country"}, inplace=True)
lci

,Country,CF for direct N emission to marine system [PDF*yr/kg]
0,Afghanistan,0.000000e+00
1,Albania,4.939992e-15
2,Algeria,4.939992e-15
3,American Samoa,0.000000e+00
4,Andorra,0.000000e+00
...,...,...
236,Western Sahara,1.288727e-15
237,Western Samoa,0.000000e+00
238,Yemen,2.680622e-15
239,Zaire,1.523601e-15


In [78]:
# Function to convert country name to ISO Alpha-2 code
def get_country_code(name):
    try:
        return pyc.countries.lookup(name).alpha_2
    except LookupError:
        try:
            return extra_mappings[name]
        except LookupError:
            print("Country code not found for ", name)
            return None  # Return None if country not found

extra_mappings = {
    "Turkey": "TR",
    "Russia": "RU",
    "Bahamas, The": "BS",
    "Brunei": "BN",
    "Byelarus": "BY",
    "Cape Verde": "CV",
    "Congo DRC": "CD",
    "Falkland Islands (Islas Malvinas)": "FK",
    "Gambia, The": "GM",
    "Ivory Coast": "CI",
    "Macedonia": "MK",
    "Macau": "MO",
    "Myanmar (Burma)": "MM",
    "Pacific Islands (Palau)": "PW",
    "Pitcairn Islands": "PN",
    "Reunion": "RE",
    "St. Vincent and the Grenadines": "VC",
    "St. Helena": "SH",
    "St. Kitts and Nevis": "KN",
    "St. Lucia": "LC",
    "St. Pierre and Miquelon": "PM",
    "Svalbard": "SJ",
    "Swaziland": "SZ",
    "Virgin Islands": "VI",
    "Western Samoa": "WS",
}

lci["Country_Code"] = lci["Country"].apply(get_country_code)
# remove rows with missing country codes
lci = lci.dropna(subset=["Country_Code"])
n_w_eu = 3.86913531769129E-15
luxembourg_row = pd.DataFrame({
    "Country": ["Malta"],
    "Country_Code": ["LU"],
    "CF for direct N emission to marine system [PDF*yr/kg]": [n_w_eu],
})
switzerland_row = pd.DataFrame({
    "Country": ["Switzerland"],
    "Country_Code": ["CH"],
    "CF for direct N emission to marine system [PDF*yr/kg]": [n_w_eu],
})
lci = pd.concat([lci, luxembourg_row, switzerland_row], ignore_index=True)
lci.set_index("Country_Code", inplace=True)
lci

Country code not found for  Baker Island
Country code not found for  French Southern & Antarctic Lands
Country code not found for  Gaza Strip
Country code not found for  Glorioso Islands
Country code not found for  Heard Island & McDonald Islands
Country code not found for  Howland Island
Country code not found for  Jan Mayen
Country code not found for  Jarvis Island
Country code not found for  Johnston Atoll
Country code not found for  Juan De Nova Island
Country code not found for  Man, Isle of
Country code not found for  Midway Islands
Country code not found for  Netherlands Antilles
Country code not found for  Paracel Islands
Country code not found for  South Georgia and the South Sandwich Is
Country code not found for  Spratly Islands
Country code not found for  Wake Island
Country code not found for  Zaire


,Country,CF for direct N emission to marine system [PDF*yr/kg]
Country_Code,,
AF,Afghanistan,0.000000e+00
AL,Albania,4.939992e-15
DZ,Algeria,4.939992e-15
AS,American Samoa,0.000000e+00
AD,Andorra,0.000000e+00
...,...,...
WS,Western Samoa,0.000000e+00
YE,Yemen,2.680622e-15
ZM,Zambia,0.000000e+00


In [79]:
# find if some regions are missing from lci
missing_regions = [region for region in all_regions if region not in lci.index.unique()]
print("Missing regions (lc-impact): ", missing_regions)

# find out if some regions are missing from augmented exiobase regions 
lci_regions = lci.index
missing_from_all_regions = [region for region in lci_regions if region not in all_regions]
print("Missing regions (exiobase, augmented): ", missing_from_all_regions)

# assert that all regions in lci-water are in the all_regions list
assert sorted(lci_regions) == sorted(all_regions), "Some regions in lci are not in the all_regions list"

Missing regions (lc-impact):  []
Missing regions (exiobase, augmented):  []


In [80]:
# augment dr_s_w to create dr_u_w

# new regions are calculated by dividing their corresponding row region by the number of countries in the row region
# for example, row region Argentina is sub-matrix WA divided by the number of countries in row region WA
wl = dr_s_w.loc["WL"].copy()
wl = wl / len(row_american_countries)

we = dr_s_w.loc["WE"].copy()
we = we / len(row_eu_countries)

wa = dr_s_w.loc["WA"].copy()
wa = wa / len(row_asia_pacific_countries)

wf = dr_s_w.loc["WF"].copy()
wf = wf / len(row_african_countries)

wm = dr_s_w.loc["WM"].copy()
wm = wm / len(row_middle_eastern_countries)

dr_u_w = dr_s_w.copy()
dr_u_w = dr_u_w.drop(index=row_regions.keys(), level='region')

# build a mapping of country codes to region dataframes
country_to_region = {}
for region in all_row_region_keys:
    if region in row_eu_countries:
        country_to_region[region] = we
    elif region in row_asia_pacific_countries:
        country_to_region[region] = wa
    elif region in row_african_countries:
        country_to_region[region] = wf
    elif region in row_american_countries:
        country_to_region[region] = wl
    elif region in row_middle_eastern_countries:
        country_to_region[region] = wm
    else:
        raise ValueError(f"Unknown region: {region}")

# add all new regions to dr_u_w
all_indices = []
all_data = []
for region in all_row_region_keys:
    region_data = country_to_region[region].copy()
    idx = pd.MultiIndex.from_product([[region],region_data.index], names=['region', 'sector'])
    all_indices.append(idx)
    all_data.append(region_data)

combined_idx = pd.MultiIndex.from_tuples(
    [idx for subidx in all_indices for idx in subidx]
)

combined_data = pd.concat(all_data)
combined_data.index = combined_idx

dr_u_w = pd.concat([dr_u_w, combined_data])

## Calculate DR factor
Calculate the impact factors of the driver for each impact region i driven by consumption in region j product sector k.

These impact factors tell how

To calculate DR factor:
1. Calculate the monetary impact factor impact/€ from 2019 exiobase data
2. Multiply each column of DR unit with the impact factors of consumption region j in product sector k. The resulting matrix represents the distribution of impacts of 1€ consumption in each consumption region.

In [81]:
# use 2019 impact factors for calculating dr_f
# calculate dr_f - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k

dr_f_w = dr_u_w.copy()
p_agriculture_water = exio3_19.satellite.M.loc["N - agriculture - water"]
scalars_w = p_agriculture_water.to_numpy() # multipliers for each column
print(len(scalars_w))
print(dr_u_w.shape)

# multiply each column of dr_u_w by the respective column value from exio3_19.satellite_agg.M
dr_f_w = dr_f_w * scalars_w # same as dr_f_w * diag(scalars) but more efficient with numpy broadcasting
dr_f_w

9800
(45000, 9800)


AT                \
                                                     Paddy rice         Wheat   
AT Paddy rice                                               0.0  0.000000e+00   
   Wheat                                                    0.0  9.106722e+03   
   Cereal grains nec                                        0.0  3.529584e-01   
   Vegetables, fruit, nuts                                  0.0  1.747251e-02   
   Oil seeds                                                0.0  8.927967e-02   
...                                                         ...           ...   
YE Membership organisation services n.e.c. (91)             0.0  8.214317e-06   
   Recreational, cultural and sporting services (92)        0.0  2.857077e-04   
   Other services (93)                                      0.0  1.631345e-05   
   Private households with employed persons (95)            0.0  8.743153e-08   
   Extra-territorial organizations and bodies               0.0  0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  1.994812e-01   
   Cereal grains nec                                      9.777910e+03   
   Vegetables, fruit, nuts                                1.634311e-02   
   Oil seeds                                              7.058377e-02   
...                                                                ...   
YE Membership organisation services n.e.c. (91)           4.330024e-06   
   Recreational, cultural and sporting services (92)      1.715024e-04   
   Other services (93)                                    8.839071e-06   
   Private households with employed persons (95)          4.946496e-08   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        1.800841e-01   
   Cereal grains nec                                            3.371361e-01   
   Vegetables, fruit, nuts                                      7.655917e+02   
   Oil seeds                                                    5.140892e-02   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                 2.030713e-05   
   Recreational, cultural and sporting services (92)            1.875465e-04   
   Other services (93)                                          1.895249e-05   
   Private households with employed persons (95)                4.131773e-08   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              9.301013e-02   
   Cereal grains nec                                  1.236497e-01   
   Vegetables, fruit, nuts                            7.016415e-03   
   Oil seeds                                          5.807846e+02   
...                                                            ...   
YE Membership organisation services n.e.c. (91)       1.877140e-05   
   Recreational, cultural and sporting services (92)  3.228942e-04   
   Other services (93)                                2.209263e-05   
   Private households with employed persons (95)      1.558991e-07   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                             \
             

## Calculate BDe factors (PDF/€ for each consumption region and product sector)

1. Multiply each cell of dr_u with the CF (characterisation factor from lc-impact) of the impact region to get PDF values for every entry
2. Sum up columns to get the total PDF/€ for the consumption region j product sector k

In [ ]:
# sort rows on lci in same order as dr_f.index.sortlevel
lci = lci.reindex(dr_f_w.index.get_level_values(0).unique())

# using average of P emissions to water and soil
# is this correct?
cf = lci["CF for direct N emission to marine system [PDF*yr/kg]"].to_numpy()
cf = np.repeat(cf, 200)
# expand cf_all_effects to match the shape of dr_f
cf = np.tile(cf, (dr_f_w.shape[1], 1)).T
print(cf.shape) # (37000, 9800)

# multiply each column of dr_f with cf_all_effects
pdf_w = dr_f_w * cf
pdf_w

(45000, 9800)


AT                \
                                                     Paddy rice         Wheat   
AT Paddy rice                                               0.0  0.000000e+00   
   Wheat                                                    0.0  0.000000e+00   
   Cereal grains nec                                        0.0  0.000000e+00   
   Vegetables, fruit, nuts                                  0.0  0.000000e+00   
   Oil seeds                                                0.0  0.000000e+00   
...                                                         ...           ...   
YE Membership organisation services n.e.c. (91)             0.0  2.201948e-20   
   Recreational, cultural and sporting services (92)        0.0  7.658744e-19   
   Other services (93)                                      0.0  4.373019e-20   
   Private households with employed persons (95)            0.0  2.343709e-22   
   Extra-territorial organizations and bodies               0.0  0.000000e+00   

                                                                        \
                                                     Cereal grains nec   
AT Paddy rice                                             0.000000e+00   
   Wheat                                                  0.000000e+00   
   Cereal grains nec                                      0.000000e+00   
   Vegetables, fruit, nuts                                0.000000e+00   
   Oil seeds                                              0.000000e+00   
...                                                                ...   
YE Membership organisation services n.e.c. (91)           1.160716e-20   
   Recreational, cultural and sporting services (92)      4.597331e-19   
   Other services (93)                                    2.369421e-20   
   Private households with employed persons (95)          1.325969e-22   
   Extra-territorial organizations and bodies             0.000000e+00   

                                                                              \
                                                     Vegetables, fruit, nuts   
AT Paddy rice                                                   0.000000e+00   
   Wheat                                                        0.000000e+00   
   Cereal grains nec                                            0.000000e+00   
   Vegetables, fruit, nuts                                      0.000000e+00   
   Oil seeds                                                    0.000000e+00   
...                                                                      ...   
YE Membership organisation services n.e.c. (91)                 5.443575e-20   
   Recreational, cultural and sporting services (92)            5.027412e-19   
   Other services (93)                                          5.080445e-20   
   Private households with employed persons (95)                1.107572e-22   
   Extra-territorial organizations and bodies                   0.000000e+00   

                                                                    \
                                                         Oil seeds   
AT Paddy rice                                         0.000000e+00   
   Wheat                                              0.000000e+00   
   Cereal grains nec                                  0.000000e+00   
   Vegetables, fruit, nuts                            0.000000e+00   
   Oil seeds                                          0.000000e+00   
...                                                            ...   
YE Membership organisation services n.e.c. (91)       5.031903e-20   
   Recreational, cultural and sporting services (92)  8.655573e-19   
   Other services (93)                                5.922199e-20   
   Private households with employed persons (95)      4.179066e-22   
   Extra-territorial organizations and bodies         0.000000e+00   

                                                                             \
             

In [83]:
# sum up all columns to get the total impact for each region-sector pair
pdf_total_w = pdf_w.sum()
pdf_total_w

AT  Paddy rice                                           0.000000e+00
    Wheat                                                8.371599e-11
    Cereal grains nec                                    1.880606e-11
    Vegetables, fruit, nuts                              4.764173e-12
    Oil seeds                                            9.244339e-11
                                                             ...     
WM  Membership organisation services n.e.c. (91)         8.024792e-13
    Recreational, cultural and sporting services (92)    1.188677e-12
    Other services (93)                                  1.284846e-12
    Private households with employed persons (95)        9.223753e-13
    Extra-territorial organizations and bodies           0.000000e+00
Length: 9800, dtype: float64

In [84]:
pd.DataFrame(pdf_total_w).to_csv("csv/pdf-eutrophication-marine-water.csv", index=True)